# Automatic Evaluator

In [12]:
%%capture
#########################
# Requirements and Imports
#########################
!pip install ipywidgets
!pip install widgetsnbextension

from os import path
import ipywidgets as widgets
from pathlib import Path

import torch
import numpy as np
import random

In [13]:
reproducibility_field = widgets.Checkbox(
    value = True,
    description = "Reproducibility",
    disabled = False
)
display(reproducibility_field)

Checkbox(value=True, description='Reproducibility')

In [14]:
reproducibility = reproducibility_field.value
if reproducibility:
    seed_field = widgets.IntText(
        value=0,
        placeholder=0,
        description="Seed:",
        disabled=False
    )
    display(seed_field)

IntText(value=0, description='Seed:')

In [15]:
# Set seeds for PRGs

reproducibility = reproducibility_field.value
if reproducibility:
    seed = seed_field.value
    print(seed)
    
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Use deterministic Algorithms
    torch.use_deterministic_algorithms(True)
    torch.utils.deterministic.fill_uninitialized_memory = True

0


## Game Selection

In [16]:
import ocatari
all_games = ocatari.core.AVAILABLE_GAMES
print(all_games)

['Adventure', 'AirRaid', 'Alien', 'Amidar', 'Assault', 'Asterix', 'Asteroids', 'Atlantis', 'BankHeist', 'BattleZone', 'BeamRider', 'Berzerk', 'Bowling', 'Boxing', 'Breakout', 'Carnival', 'Centipede', 'ChopperCommand', 'CrazyClimber', 'DemonAttack', 'DonkeyKong', 'DoubleDunk', 'Enduro', 'FishingDerby', 'Freeway', 'Frogger', 'Frostbite', 'Galaxian', 'Gopher', 'Hero', 'IceHockey', 'Jamesbond', 'Kangaroo', 'KeystoneKapers', 'KingKong', 'Krull', 'KungFuMaster', 'MarioBros', 'MontezumaRevenge', 'MsPacman', 'NameThisGame', 'Pacman', 'Phoenix', 'Pitfall', 'Pong', 'Pooyan', 'PrivateEye', 'Qbert', 'Riverraid', 'RoadRunner', 'Seaquest', 'Skiing', 'SpaceInvaders', 'StarGunner', 'Tennis', 'TimePilot', 'UpNDown', 'Venture', 'VideoPinball', 'YarsRevenge', 'Zaxxon']


In [17]:
hack_games = [x.name.replace('.py', '') for x in Path('../hackatari/games').glob('*.py') if x.is_file() and x.name != '__init__.py']

games_option = []
for g in all_games:
    if g.lower() in hack_games:
        games_option.append(g)


game_selector = widgets.Select(
    options = games_option,
    value = "Pong",
    description = 'Game:',
    disabled = False,
)
game_selector.layout.width = '30%'
display(game_selector)

Select(description='Game:', index=21, layout=Layout(width='30%'), options=('Amidar', 'Assault', 'Asterix', 'At…

In [8]:
selected_game = game_selector.value
selected_game

'Pong'

# Model Selection

In [ ]:
from IPython.display import display, clear_output

model_dir_input = widgets.Text(
    value='../models',
    placeholder='Enter relative path...',
    description='Relative Path:',
    disabled=False,
    layout={'width': '30%'}
)

output_area = widgets.Output() #create an output area for feedback

def on_button_click(b):
    global stored_path_object
    with output_area: #capture output into the defined output area
        clear_output() #clear previous output
        relative_path = model_dir_input.value
        
        if os.path.exists(relative_path):
            print(f"Path '{relative_path}' exists. Path is stored.")
            stored_path_object = Path(relative_path) #create path object
        else:
            print(f"Path '{relative_path}' does not exist. Path is set to None.")
            stored_path_object = None
              

button = widgets.Button(description="Verify Model Path")
button.on_click(on_button_click)

input_and_button = widgets.HBox([model_dir_input, button]) # create hbox

# Display the widget
display(input_and_button, output_area)


Output()

In [58]:
models = [m for m in stored_path_object.glob('**/*.gz') if m.is_file() and selected_game.lower() in str(m).lower()]

model_selector = widgets.SelectMultiple(
    options=models,
    value=[],
    description="Select Models",
    display="flex",
    flew_flow="column",
    align_items="stretch",
    style={"description_width": "initial"},
    disabled=False
)
model_selector.layout.width = '30%'
display(model_selector)


AttributeError: 'NoneType' object has no attribute 'glob'

In [45]:
models_list = list(model_selector.value)
assert models_list, "Please select at least one model"
models_list

[PosixPath('../models/Pong/0/c51_classic_50M.gz')]

# Add Run Arguments

In [46]:
import hackatari.core as core

avail_mod_for_game = core._available_modifications(selected_game).split('*')[1:]
modifies_list = [mod.split(':') for mod in avail_mod_for_game]

modifies_selector = widgets.VBox([
        widgets.HBox([
            widgets.Label(value=mod_name, layout={'width': '10%'}),
            widgets.Label(value=mod_info, layout={'width': '50%'}),
            widgets.Dropdown(options=[True, False], value=False, description='Value:',layout={'width': '15%'}, justify_content = 'flex-end')
        ], justify_content = 'space-between') for mod_name, mod_info in modifies_list
    ], layout={'width': '75%'})

display(modifies_selector)

In [47]:
mod_args = ' '.join([f'{mod.children[0].value.strip()}' for mod in modifies_selector.children if mod.children[2].value])
mod_args

'left_up_drift slow_ball'

# Run Game

In [48]:
## prepare the command
eval_script = Path(r"scripts/eval.py")
run_script = Path(r"scripts/run.py")

notebook_dir = path.abspath("")
project_root = path.dirname(notebook_dir)
print(f"Project Root: {project_root}")

# use posix_paths compatibile with the OS
eval_path = (Path(project_root) / eval_script).as_posix()
run_path = (Path(project_root) / run_script).as_posix()

models_path = ' '.join( f'{model.as_posix()}' for model in models_list )


command = f"python {eval_path} -g {selected_game} -a {models_path} -m {mod_args}"

print("command: " + command)


Project Root: /mnt/c/Users/peng_/workspace/HackAtari
command: python /mnt/c/Users/peng_/workspace/HackAtari/scripts/eval.py -g Pong -a ../models/Pong/0/c51_classic_50M.gz -m left_up_drift slow_ball


In [49]:
# execute the command
!{command}

A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
Loaded agent from ../models/Pong/0/c51_classic_50M.gz
Episode 1: Reward = -21.0
Episode 2: Reward = -21.0
Episode 3: Reward = -21.0
Episode 4: Reward = -21.0
Episode 5: Reward = -20.0
Episode 6: Reward = -21.0
Episode 7: Reward = -20.0
Episode 8: Reward = -21.0
Episode 9: Reward = -21.0
Episode 10: Reward = -21.0

Summary:
Agent: ../models/Pong/0/c51_classic_50M.gz
Total Episodes: 10
Average Reward: -20.80
Standard Deviation: 0.40
Min Reward: -21.0
Max Reward: -20.0
--------------------------------------
------------------------------------------------
Overall Average Reward: -20.80
Overall Standard Deviation: 0.40
------------------------------------------------


# Evaluation

In [ ]:
# add automatic_evaluator module to the path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

## Test with all subsets of selected modifs

In [50]:
from itertools import combinations
mod_list = mod_args.split(" ")

#mod_list = ['A','B','C']

# return all non-emtpy subsets
def subsets(fullset):    
    result = []
    for i in range(1, len(fullset) + 1):
        tmp = [ list(c) for c in combinations(fullset, i) ]
        result += tmp
        
    return result

mod_sublists = subsets(mod_list)

print(mod_sublists)

[['left_up_drift'], ['slow_ball'], ['left_up_drift', 'slow_ball']]


In [51]:
%reload_ext aim

!aim init -y # it will re-initialize empty aim repo.

%aim up

Re-initialized empty Aim repository at /mnt/c/Users/peng_/workspace/HackAtari/automatic_evaluator


In [52]:
%reload_ext autoreload

%autoreload 1

%aimport utils

# agent_paths = [model.as_posix() for model in models_list]
# for agent_path in agent_paths:
#     agent_name = "/".join(agent_path.split("/")[-2:])
#     print(agent_name)
        
for modifs in mod_sublists:
    utils.eval_withAimRun(selected_game, [ model.as_posix() for model in models_list], modifs)


A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]


Loaded agent from ../models/Pong/0/c51_classic_50M.gz
--------------------------------------
Done.
Loaded agent from ../models/Pong/0/c51_classic_50M.gz
--------------------------------------
Done.
Loaded agent from ../models/Pong/0/c51_classic_50M.gz


KeyboardInterrupt: 